# Scoring

In this notebook we use the code we have implemented to assess the saftey based on rules 1 and 2 of RSS

In [ ]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes, NuScenesExplorer

import numpy as np

from minf_part1_functions import *
from my_nuscenes_functions import *

dataroot = 'data/sets/nuscenes'

nusc = NuScenes(version='v1.0-mini', dataroot=dataroot, verbose=False)

scene = nusc.scene[0]
sample = nusc.get('sample', scene['first_sample_token'])

def get_score(score_dict):
    return score_dict['score']

scores = []
instances = []
done = False
# loop through all samples in scenes
while not done:
    # loop through all annotations in sample
    for ann in sample['anns']:
        instance_token = nusc.get('sample_annotation', ann)['instance_token']
        instance = nusc.get('instance', instance_token)
        category = nusc.get('category', instance['category_token'])['name']
        if instance_token not in instances and 'vehicle' in category:
            s = generate_scores_for_instance(nusc, instance_token)
            s = min(s, key=get_score)
            s['instance'] = instance_token
            scores.append(s)
            instances.append(instance_token)
    
    # check for next sample
    if sample['next']:
        sample = nusc.get('sample', sample['next'])
    else:
        done = True

In [ ]:
[score for score in scores if score['score'] < 1]